<a href="https://colab.research.google.com/github/venkatanadikatla/pytorch/blob/main/RNN_Building_Training_and_Evaluation_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y torchtext
!pip install torchtext==0.6.0

Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached n

In [2]:
import copy # this module provides functions to duplicate objects. It seems to be imported but not yet used in the code
import torch # The MAIN PyTorch package
from torch import nn # contains the essential modules for building NN in pytorch.
from torch import optim # Provides optimization algorithms, such as SGD, Adam, etc
import torchtext # A library for text processing that works well with pytorch (Currently a version 0.6.0 is being used in this code)
from torchtext import data # A module in torchtext used for data handling
from torchtext import datasets # provides datasets, including various NLP datasets.

TEXT = data.Field(sequential=True, batch_first=True, lower=True) # Sequential = True indicates that the data consists of sequences.
#Batch_first=True Ensure that batch dimenstion is the first dimension in the tensor. # lower=True Converts all the text to lowercase

LABEL = data.LabelField() # A subclass of Field specifically for handling labels in a classification task.

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL) #datasets.SST.splits - Loads the Standford Sentiment Treebank(SST) dataset and splits the dataset

# build dictionary
# build_vocab: Creates a mapping from tokens(words) to indices. This is essential for converting text data into numerical form that can be used by NN.
TEXT.build_vocab(train_data) # Builds the vocabulary for the text field using the training data.
LABEL.build_vocab(train_data)# Builds the vocabulary for the label field using the training data.

# hyperparameters
vocab_size = len(TEXT.vocab) # the size of the vocabulary (number of unique tokens in the training data)
label_size = len(LABEL.vocab) # the number of unique labels (classes) in the traning data
padding_idx = TEXT.vocab.stoi['<pad>'] # The index used for padding sequences to the same length
embedding_dim = 128 # The size of the word embeddings (dense vector representation of words)
hidden_dim = 128 # Size of the hidden layers in the model

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=32)

# Data.bucketiterator.splits - Creates iterators for the training, validation and test sets.


downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:02<00:00, 309kB/s]


extracting


In [3]:
print(vars (train_data.examples[0]))

{'text': ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], 'label': 'positive'}


In [4]:
print(vars(val_data.examples[0]))

{'text': ['it', "'s", 'a', 'lovely', 'film', 'with', 'lovely', 'performances', 'by', 'buy', 'and', 'accorsi', '.'], 'label': 'positive'}


In [5]:
print(vars(test_data.examples[0]))

{'text': ['effective', 'but', 'too-tepid', 'biopic'], 'label': 'neutral'}


In [6]:
print(f'Num Train: {len(train_data)}')
print(f'Num Val: {len(val_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 8544
Num Val: 1101
Num Test: 2210


In [7]:
print(f'Vocabulary size: {len(TEXT.vocab)}')

Vocabulary size: 16581


In [8]:
print(f'Vocabulary size: {len(LABEL.vocab)}')

Vocabulary size: 3


In [9]:
print(LABEL.vocab.freqs)


Counter({'positive': 3610, 'negative': 3310, 'neutral': 1624})


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 8024), ('the', 7303), (',', 7131), ('a', 5281), ('and', 4473), ('of', 4396), ('to', 3021), ('is', 2561), ("'s", 2544), ('it', 2422), ('that', 1954), ('in', 1888), ('as', 1296), ('but', 1172), ('film', 1162), ('with', 1139), ('for', 1023), ('this', 998), ('movie', 976), ('an', 972)]


In [11]:
print(TEXT.vocab.itos[:10])
#Tokens corresponding to the first 10 indices (0,1....9)

['<unk>', '<pad>', '.', 'the', ',', 'a', 'and', 'of', 'to', 'is']


In [12]:
class RNN (torch.nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx):
    super().__init__()
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.label_size = label_size
    self.num_layers = 1
    self.embedding = torch.nn.Embedding(vocab_size, embedding_dim, padding_idx = padding_idx)
    # self.rnn = torch.nn.RNN(embedding_dim,hidden_dim, nonlinearity='relu',batch_first=True)
    self.lstm = torch.nn.LSTM(embedding_dim,hidden_dim, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, output_dim)

  def zero_state(self, batch_size):
    # Implement the function, which returns an initial hidden state.
    return torch.zeros(self.num_layers, batch_size, self.hidden_dim)


  def forward(self, text):
    #text dim = [sentence length, batchsize]
    embedded = self.embedding(text)
    #embedded dim = [sentence length, batchsize, embedding_dim]
    batch_size = text.size(0)
    h_0 = self.zero_state(batch_size).to(text.device)  # Ensure the hidden state is on the same device as the input
    output, (hidden,cell) = self.lstm(embedded)
    #output dim = [sentence length, batchsize, hidden_dim]
    #hidden dim = [1, batch_size, hidden_dim]
    hidden = hidden.squeeze_(0)
    #hidden_dim = [batch_size, hidden_dim]

    return self.fc(hidden)





In [13]:
def train_model(model,train_iter, optimizer, criterion, num_epochs =10):
  model.train()
  for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    for batch in train_iter:
      optimizer.zero_grad()
      text, labels = batch.text, batch.label
      output = model(text)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) ==labels).sum().item()

      _, predicted = torch.max(output.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    avg_loss = epoch_loss/len(train_iter)
    avg_acc = epoch_acc/len(train_iter.dataset)

    epoch_accuracy = 100*correct/total


    print(f' Epoch {epoch+1}, Train Loss: {avg_loss}, Train Accuracy: {epoch_accuracy}%')



In [14]:
def eval_model(model, val_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in val_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(val_iter)
  avg_acc = epoch_acc/len(val_iter.dataset)

  epoch_accuracy = 100*correct/total

  print(f'Validation Loss: {avg_loss},  Validation Accuracy: {epoch_accuracy}%')




In [15]:
def test_model(model, test_iter, criterion):
  model.eval()
  epoch_loss = 0
  epoch_acc = 0
  correct = 0
  total = 0
  for batch in test_iter:
    text, labels = batch.text, batch.label
    output = model(text)
    loss = criterion(output, labels)
    epoch_loss +=loss.item()
    epoch_acc += (output.argmax(1)==labels).sum().item()
    _, predicted = torch.max(output.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  avg_loss = epoch_loss/len(test_iter)
  avg_acc = epoch_acc/len(test_iter.dataset)

  epoch_accuracy = 100*correct/total

  print(f'Test Loss: {avg_loss},  Test Accuracy: {epoch_accuracy}%')




In [16]:
torch.manual_seed(42)
model = RNN(vocab_size,embedding_dim,hidden_dim,label_size, padding_idx)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [17]:
train_model(model,train_iter, optimizer, criterion, num_epochs =10)

 Epoch 1, Train Loss: 1.0503627456975786, Train Accuracy: 41.61985018726592%
 Epoch 2, Train Loss: 1.0456770968794376, Train Accuracy: 42.16994382022472%
 Epoch 3, Train Loss: 1.0410769454548867, Train Accuracy: 42.75514981273408%
 Epoch 4, Train Loss: 1.0306991256131661, Train Accuracy: 43.56273408239701%
 Epoch 5, Train Loss: 0.9933238547392998, Train Accuracy: 50.50327715355805%
 Epoch 6, Train Loss: 0.9218269372254275, Train Accuracy: 58.26310861423221%
 Epoch 7, Train Loss: 0.8122221905640449, Train Accuracy: 66.0814606741573%
 Epoch 8, Train Loss: 0.7005944786670056, Train Accuracy: 70.7748127340824%
 Epoch 9, Train Loss: 0.5846632567237825, Train Accuracy: 75.51498127340824%
 Epoch 10, Train Loss: 0.49156253173779907, Train Accuracy: 78.6750936329588%


In [18]:
# Evaluate the model on validation data
eval_model(model, val_iter, criterion)



Validation Loss: 1.2625662888799394,  Validation Accuracy: 54.58673932788374%


In [19]:
# Evaluate the model on test data
test_model(model, test_iter, criterion)

Test Loss: 1.2099011038030898,  Test Accuracy: 58.23529411764706%
